In [1]:
import os

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, expr
from delta.tables import DeltaTable
from delta import configure_spark_with_delta_pip
import shutil

# https://docs.delta.io/latest/index.html

# delta tables are an addon for spark to be able to have sql-like features for data stored in your data lake (acid-compliant, history of actions on the table etc).
# it uses versioned, immutable parquet files to store data in cloud storage (s3), and keeps a transaction log.
# have to specify storage location.  can copy data from 1 location to another.  can stream directly to a delta table.
# portable code with other spark platforms.

# i dont think this is worth exploring in any further depth unless a role revolves around using spark almost exclusively.

# this Clears any previous runs
shutil.rmtree("/tmp/delta-table", ignore_errors=True)

conf = SparkConf()
conf.set('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.1') # basically most recent version as my spark version
conf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
conf.set('spark.hadoop.fs.s3a.access.key', os.environ.get('aws_access_key_id'))
conf.set('spark.hadoop.fs.s3a.secret.key', os.environ.get('aws_secret_access_key'))
conf.set('spark.sql.extensions', "io.delta.sql.DeltaSparkSessionExtension")
conf.set("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

builder = SparkSession.builder.appName("MyApp") \
    .config(conf = conf)

spark = configure_spark_with_delta_pip(builder).getOrCreate()

22/05/20 16:03:15 WARN Utils: Your hostname, jacob-BigOtisLinux resolves to a loopback address: 127.0.1.1; using 192.168.50.142 instead (on interface enp6s0)
22/05/20 16:03:15 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/jacob/.ivy2/cache
The jars for the packages stored in: /home/jacob/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bdac31cf-abf1-4ca1-b295-72ff26a8e6dc;1.0
	confs: [default]
	found io.delta#delta-core_2.12;1.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central


:: loading settings :: url = jar:file:/home/jacob/.local/share/virtualenvs/pyspark_prac-QhYfwHaC/lib/python3.8/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 83ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;1.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3   |   0   |   0   |   0   ||   3   |   0   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-bdac31cf-abf1-4ca1-b295-72ff26a8e6dc
	confs: [default]
	0 artifacts copied, 3 already retrieved (0kB/3ms)
22/05/20 16:03:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your p

In [2]:
print("############# Creating a table ###############")
data = spark.range(0, 5)
data.write.format("delta").save("/tmp/delta-table")



############# Creating a table ###############


In [3]:
print("############ Reading the table ###############")
df = spark.read.format("delta").load("/tmp/delta-table")
df.show()

############ Reading the table ###############
+---+
| id|
+---+
|  4|
|  3|
|  2|
|  0|
|  1|
+---+



In [4]:
print("########### Upsert new data #############")
newData = spark.range(0, 20)

########### Upsert new data #############


In [5]:
deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

In [6]:
deltaTable.alias("oldData")\
    .merge(
    newData.alias("newData"),
    "oldData.id = newData.id")\
    .whenMatchedUpdate(set={"id": col("newData.id")})\
    .whenNotMatchedInsert(values={"id": col("newData.id")})\
    .execute()


In [7]:
deltaTable.toDF().show()

+---+
| id|
+---+
|  0|
|  1|
|  2|
|  3|
|  4|
|  5|
|  6|
|  7|
|  8|
|  9|
| 10|
| 11|
| 12|
| 13|
| 14|
| 15|
| 16|
| 17|
| 18|
| 19|
+---+



In [8]:
print("########## Overwrite the table ###########")
data = spark.range(5, 10)
data.write.format("delta").mode("overwrite").save("/tmp/delta-table")
deltaTable.toDF().show()

deltaTable = DeltaTable.forPath(spark, "/tmp/delta-table")

########## Overwrite the table ###########
+---+
| id|
+---+
|  5|
|  9|
|  7|
|  6|
|  8|
+---+



In [9]:
print("########### Update to the table(add 100 to every even value) ##############")
deltaTable.update(
    condition=expr("id % 2 == 0"),
    set={"id": expr("id + 100")})

deltaTable.toDF().show()


########### Update to the table(add 100 to every even value) ##############
+---+
| id|
+---+
|  5|
|  9|
|  7|
|108|
|106|
+---+



In [10]:
print("######### Delete every even value ##############")
deltaTable.delete(condition=expr("id % 2 == 0"))
deltaTable.toDF().show()

# Read old version of data using time travel
print("######## Read old data using time travel ############")
df = spark.read.format("delta").option("versionAsOf", 0).load("/tmp/delta-table")
df.show()


######### Delete every even value ##############
+---+
| id|
+---+
|  5|
|  9|
|  7|
+---+

######## Read old data using time travel ############
+---+
| id|
+---+
|  4|
|  1|
|  2|
|  0|
|  3|
+---+



In [4]:
df.write.format("delta").mode('overwrite').save("s3a://jacobsbucket97/sample_files/delta_table1.delta")

Py4JJavaError: An error occurred while calling o62.save.
: java.lang.RuntimeException: java.lang.ClassNotFoundException: Class org.apache.hadoop.fs.s3a.S3AFileSystem not found
	at org.apache.hadoop.conf.Configuration.getClass(Configuration.java:2667)
	at org.apache.hadoop.fs.FileSystem.getFileSystemClass(FileSystem.java:3431)
	at org.apache.hadoop.fs.FileSystem.createFileSystem(FileSystem.java:3466)
	at org.apache.hadoop.fs.FileSystem.access$300(FileSystem.java:174)
	at org.apache.hadoop.fs.FileSystem$Cache.getInternal(FileSystem.java:3574)
	at org.apache.hadoop.fs.FileSystem$Cache.get(FileSystem.java:3521)
	at org.apache.hadoop.fs.FileSystem.get(FileSystem.java:540)
	at org.apache.hadoop.fs.Path.getFileSystem(Path.java:365)
	at org.apache.spark.sql.delta.DeltaLog$.apply(DeltaLog.scala:546)
	at org.apache.spark.sql.delta.DeltaLog$.forTable(DeltaLog.scala:458)
	at org.apache.spark.sql.delta.sources.DeltaDataSource.createRelation(DeltaDataSource.scala:147)
	at org.apache.spark.sql.execut

In [11]:
# cleanup
shutil.rmtree("/tmp/delta-table")